In [1]:
import pandas as pd, datetime as dt
from datetime import timedelta, datetime, date
import numpy as np
import xlwings as xw
import os
import openpyxl
from openpyxl import load_workbook
from openpyxl.utils.dataframe import dataframe_to_rows
import warnings
import matplotlib.pyplot as plt
warnings.filterwarnings("ignore")

In [5]:
# Define la URL del archivo CSV en GitHub
url = "https://raw.githubusercontent.com/lucaschicco/Remo/master/fusion%20calientes.csv"

# Lee el archivo CSV en un DataFrame de pandas
fus2 = pd.read_csv(url, delimiter=",")
fus2.rename(columns={'SiteNumber':'PBL','Site':'Nombre','Name':'Producto'},inplace=True)
#ss2.drop(columns='Name',inplace=True)
fus = fus2
# Convertir la columna "Fecha" al tipo de dato datetime
fus['Fecha'] = pd.to_datetime(fus['Fecha'])

# Agregar la columna "Dia_Semana" que indica el nombre del día de la semana
fus['Dia_Semana'] = fus['Fecha'].dt.day_name()

# Obtener el año de la fecha de inicio
inicio = fus['Fecha'].min().year

# Agregar la columna "Semana" que indica el número de semana del año
condiciones = [
    (fus['Fecha'].dt.year == 2022),
    (fus['Fecha'].dt.year == 2023)
]
opciones = [
    fus['Fecha'].dt.isocalendar().week,
    fus['Fecha'].dt.isocalendar().week + 52
]
fus['Semana'] = np.select(condiciones, opciones, default=0)

# Ajustar el número de semana para los domingos (Sunday)
fus.loc[fus['Dia_Semana'] == 'Sunday', 'Semana'] += 1

# Establecer manualmente el número de semana para la fecha "2023-01-01"
fus.loc[fus['Fecha'] == '2022-01-01', 'Semana'] = 1
fus.loc[fus['Fecha'] == '2022-01-02', 'Semana'] = 1
fus.loc[fus['Fecha'] == '2023-01-01', 'Semana'] = 53
# Definir el orden deseado de los días de la semana
orden_dias_semana = ['Sunday','Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday']

# Crear una columna categórica con el orden deseado
fus['Dia_Semana'] = pd.Categorical(fus['Dia_Semana'], categories=orden_dias_semana, ordered=True)
fus['Fecha'] = fus['Fecha'].dt.date
fus['Fecha'] = pd.to_datetime(fus['Fecha'])

In [3]:
fus.sort_values(by='Fecha', inplace=True)
fus3 = fus.drop(columns=['Fecha','Dia_Semana'])
fus4 = fus3.groupby(['PBL','Producto','Semana','Nombre']).sum().reset_index()


In [4]:
import pandas as pd
import matplotlib.pyplot as plt
import plotly.express as px
from ipywidgets import interact, widgets

# Convierte la columna "mes" a tipo datetime para facilitar el ordenamiento
#fus4['mes'] = pd.to_datetime(df2['mes'], format='%Y-%m')

# Define una paleta de colores personalizada


# Función para crear y actualizar el gráfico interactivo
def plot_data(estacion, producto_elegido):
    filtered_df = fus4[(fus4['Nombre'] == estacion) & (fus4['Producto'] == producto_elegido)]

    fig = px.line(
            filtered_df,
            x='Semana',
            y='Volumen',
            #color='empresabandera',
            labels={'Volumen': 'Volumen'},
            title=f'Evolución del Volumen en {estacion} para {producto_elegido}'
    )


    fig.show()

# Crea tres dropdowns interactivos: uno para la localidad, otro para el tipo de visualización y otro para el producto

estaciones = fus4['Nombre'].unique()

productos = fus4['Producto'].unique()
estacion_dropdown = widgets.Dropdown(options=estaciones, description='Estación:')
#provincia_dropdown = widgets.Dropdown(options=provincias, description='Provincia:')
#visualizacion_dropdown = widgets.Dropdown(options=['Market Share', 'Volumen'], description='Visualización:')
producto_dropdown = widgets.Dropdown(options=productos, description='Producto:')

interact(plot_data, estacion=estacion_dropdown,  producto_elegido=producto_dropdown)


interactive(children=(Dropdown(description='Estación:', options=('CORS LOS INCAS', 'CORS MASCHWITZ', 'EMICAS S…

<function __main__.plot_data(estacion, producto_elegido)>